In [1]:
!which python

/home/ashiktonmoy/Desktop/Winesfere_ocr/venv/bin/python


In [37]:
import pandas as pd
import os
import json
import requests
from clint.textui import progress
from time import time
from multiprocessing.pool import ThreadPool

In [18]:
from dotenv import load_dotenv
load_dotenv('env/.env') 

True

In [8]:
destination = os.path.join('data','downloaded')
print(destination)
try:
    os.makedirs(destination, exist_ok=True)
    print("Directory doesn't exist. \nCreating Directory..." )
except OSError as error:
    print(error)

data/downloaded
Directory doesn't exist. 
Creating Directory...


In [3]:
df = pd.read_excel('data/colas_sharing_form_display.xlsx', index_col=0)

,ttb_id,ct,origin,rep_id,source_of_product,address,type_of_product,mailing_address,email,formula_sop_no,...,wine_application_if_on_label,wine_vintage_date_if_on_label,phone_number,fax_number,date_of_application,print_name_of_applicant,date_issued,status,img_data,inserted_at
0,10002001000002,81,02,NaN,Domestic,VALERIE BATEMAN\n11 MOUNTAIN VIEW ESTATESRDWIN...,Wine,NaN,NaN,NaN,...,NEW YORK STATE,2008,(518) 734-5214,(518) 734-5214,2010-01-02,VALERIE BATEMAN,2010-01-06,APPROVED,"[{""img"": ""cola_images/Chardonnay_VSR2008.jpg"",...",2022-05-25 14:57:25
1,10004001000231,88,38,NaN,Domestic,"WILLAMETTE VALLEY VINEYARDS, WILLAMETTE VALLEY...",Wine,NaN,NaN,NaN,...,WILLAMETTE VALLEY,2007,(503) 588-9463,NaN,2010-01-04,JAMES BERNAU,2010-01-05,APPROVED,"[{""img"": ""cola_images/CHV_2007_pn_b_2_72x3_58_...",2022-05-25 16:35:25
2,10006001000031,80,70,NaN,Imported,INNOVINO INTERNATIONAL INC\n7707 N UNIVERSITY ...,Wine,NaN,NaN,NaN,...,CENTRAL VALLEY,2008,(305) 505-6205,(954) 839-6495,2010-01-06,DANIELLE HALL,2010-01-07,APPROVED,"[{""img"": ""cola_images/vina_d_iela_CAB_RES_bran...",2022-05-25 16:40:28
3,10008001000116,81,53,NaN,Imported,"VALCKENBERG INTERNATIONAL, INC.\n8153 E 44TH S...",Wine,NaN,NaN,NaN,...,MOSEL,2007,(815) 404-6224,NaN,2010-01-08,NATALIE RANDALL,2010-01-11,APPROVED,"[{""img"": ""cola_images/JJ4-07M.jpg"", ""text"": ""I...",2022-05-25 16:49:23
4,10011001000137,80,50,NaN,Imported,"SANTA ARMOSA, LLC\n136 SPEEDWELL AVEMORRISTOWN...",Wine,NaN,NaN,NaN,...,SALENTO\nINDICAZIONE GEOGRAFICA TIPICA,2008,(866) 709-7289,NaN,2010-01-11,GAL KEREN,2010-01-15,APPROVED,"[{""img"": ""cola_images/NeroDiMoinsignoreFinale....",2022-05-25 16:54:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,14233001000224,88,01,NaN,Domestic,TREASURY WINE ESTATES AMERICAS COMPANY\n1250 C...,Wine,NaN,NaN,NaN,...,OAKVILLE,2012,(707) 299-3142,NaN,2014-08-21,Sarah Spiller,2014-09-13,APPROVED,"[{""img"": ""cola_images/2012_ETU_Vine_Hill_CabS_...",2022-05-31 15:18:56
1723,14233001000254,88,51,NaN,Imported,"WINEBOW, BOSTON WINE COMPANY, LTD.\n30 DANE ST...",Wine,NaN,NaN,NaN,...,MEDOC,2009,(201) 930-2392,(201) 930-2413,2014-08-21,PATRICIA PATTERSON,2014-09-15,APPROVED,"[{""img"": ""cola_images/Goulee_front_1.jpg"", ""te...",2022-05-31 15:17:30
1724,14239001000236,88,01,NaN,Domestic,"FREE FLOW WINES, FREE FLOW WINES, LLC\n2557 NA...",Wine,NaN,NaN,NaN,...,SONOMA VALLEY,2013,(415) 626-1215,(415) 373-3925,2014-08-27,TRACY KASIN,2014-09-16,APPROVED,"[{""img"": ""cola_images/2013_Custard_CH.jpg"", ""t...",2022-05-31 15:42:01
1725,14240001000113,81,52,NaN,Imported,"VINE PRO IMPORTS, LLC\n7535 W 24TH AVE , # S-1...",Wine,NaN,NaN,NaN,...,VINO DE LA TIERRA DE CASTILLA Y LEON,2012,(786) 546-0551,NaN,2014-08-28,Tiffany Gonzalez,2014-09-24,APPROVED,"[{""img"": ""cola_images/PLwhiteFrontNEW.JPG"", ""t...",2022-05-31 15:45:59


In [4]:
df.head()

,ttb_id,ct,origin,rep_id,source_of_product,address,type_of_product,mailing_address,email,formula_sop_no,...,wine_application_if_on_label,wine_vintage_date_if_on_label,phone_number,fax_number,date_of_application,print_name_of_applicant,date_issued,status,img_data,inserted_at
0,10002001000002,81,02,NaN,Domestic,VALERIE BATEMAN\n11 MOUNTAIN VIEW ESTATESRDWIN...,Wine,NaN,NaN,NaN,...,NEW YORK STATE,2008,(518) 734-5214,(518) 734-5214,2010-01-02,VALERIE BATEMAN,2010-01-06,APPROVED,"[{""img"": ""cola_images/Chardonnay_VSR2008.jpg"",...",2022-05-25 14:57:25
1,10004001000231,88,38,NaN,Domestic,"WILLAMETTE VALLEY VINEYARDS, WILLAMETTE VALLEY...",Wine,NaN,NaN,NaN,...,WILLAMETTE VALLEY,2007,(503) 588-9463,NaN,2010-01-04,JAMES BERNAU,2010-01-05,APPROVED,"[{""img"": ""cola_images/CHV_2007_pn_b_2_72x3_58_...",2022-05-25 16:35:25
2,10006001000031,80,70,NaN,Imported,INNOVINO INTERNATIONAL INC\n7707 N UNIVERSITY ...,Wine,NaN,NaN,NaN,...,CENTRAL VALLEY,2008,(305) 505-6205,(954) 839-6495,2010-01-06,DANIELLE HALL,2010-01-07,APPROVED,"[{""img"": ""cola_images/vina_d_iela_CAB_RES_bran...",2022-05-25 16:40:28
3,10008001000116,81,53,NaN,Imported,"VALCKENBERG INTERNATIONAL, INC.\n8153 E 44TH S...",Wine,NaN,NaN,NaN,...,MOSEL,2007,(815) 404-6224,NaN,2010-01-08,NATALIE RANDALL,2010-01-11,APPROVED,"[{""img"": ""cola_images/JJ4-07M.jpg"", ""text"": ""I...",2022-05-25 16:49:23
4,10011001000137,80,50,NaN,Imported,"SANTA ARMOSA, LLC\n136 SPEEDWELL AVEMORRISTOWN...",Wine,NaN,NaN,NaN,...,SALENTO\nINDICAZIONE GEOGRAFICA TIPICA,2008,(866) 709-7289,NaN,2010-01-11,GAL KEREN,2010-01-15,APPROVED,"[{""img"": ""cola_images/NeroDiMoinsignoreFinale....",2022-05-25 16:54:47


In [10]:
# # img_data
# df.index

In [43]:
urlHead = os.environ.get("urlHead")
for entry in df.index:
    dest = os.path.join(destination, str(entry))  # destination path
    

    if pd.isna(entry) == False:
        img_data = json.loads(df.loc[entry, 'img_data'])
        if os.path.exists(dest) == False:
            try:
                os.makedirs(dest, exist_ok=True)
#                 print(f"Directory doesn't exist. \nCreating Directory: {dest} ")
            except OSError as error:
                print(error)

        for images in img_data:
            start = time()
            
            #actual url of image
            url= os.path.join( urlHead , images['img']) 
            
            # get the image name
            head_tail = os.path.split(url)               
            frame_name = os.path.join(dest, head_tail[1])
            
            
            # download the file
            r = requests.get(url, stream = True)
            
            # save the file
            with open(frame_name, "wb") as f:
                total_length = int(r.headers.get('content-length'))
                
                #for progresss bar in terminal
                for chunk in progress.bar(r.iter_content(chunk_size=2391975), expected_size=(total_length/1024) + 1):
                    if chunk:
                        f.write(chunk)
            print(f"{frame_name}: took time: {time() - start :.3f}s")


data/downloaded/0/Chardonnay_VSR2008.jpg: took time: 0.087s
data/downloaded/0/backlabel.jpg: took time: 0.794s
data/downloaded/1/CHV_2007_pn_b_2_72x3_58_14_5.jpg: took time: 1.302s
data/downloaded/1/CHV_2007_pn_f_4_68x2_98.jpg: took time: 0.832s
data/downloaded/2/vina_d_iela_CAB_RES_brand_label.jpg: took time: 0.098s
data/downloaded/3/JJ4-07M.jpg: took time: 0.185s
data/downloaded/4/NeroDiMoinsignoreFinale.jpg: took time: 0.216s
data/downloaded/4/NERO_DI_MONSIGNORE-back.jpg: took time: 0.860s
data/downloaded/5/ZOOM_Niagara.jpg: took time: 0.754s
data/downloaded/6/shiloh_cabernet_sauv_north_coast_front.jpg: took time: 0.960s
data/downloaded/6/shiloh_cabernet_sauv_north_coast_back.jpg: took time: 1.014s
data/downloaded/7/chbrandPort07.jpg: took time: 1.036s
data/downloaded/7/chandbackport07.jpg: took time: 0.834s
data/downloaded/8/gi_temp_brand_11.jpg: took time: 0.984s
data/downloaded/8/gitempback4.jpg: took time: 0.811s
data/downloaded/9/Front_Brand_Label.jpg: took time: 1.234s
data/do

data/downloaded/65/PN_08_front.jpg: took time: 0.991s
data/downloaded/66/FL_side2.jpg: took time: 0.822s
data/downloaded/66/FL_top.jpg: took time: 0.858s
data/downloaded/67/3104695401.jpg: took time: 1.951s
data/downloaded/67/Acacia_Generic_Back.jpg: took time: 0.834s
data/downloaded/68/Paradisos-del-Sol-Triangle-.jpg: took time: 0.743s
data/downloaded/68/2008-Rose-Paradiso-Front.jpg: took time: 1.494s
data/downloaded/68/2008-Rose-Paradiso-Back.jpg: took time: 0.780s
data/downloaded/69/375-Renesium-brand-4.jpg: took time: 3.666s
data/downloaded/69/RENESIUM.JPG: took time: 1.047s
data/downloaded/70/375_front.JPG: took time: 0.756s
data/downloaded/70/375_back.JPG: took time: 0.986s
data/downloaded/71/Donjon_GranTrad_Brand_1.jpg: took time: 1.470s
data/downloaded/71/Donjon_GT_Brand_2_rev.jpg: took time: 0.777s
data/downloaded/71/Donjon_GT_Body_2_Rev.jpg: took time: 0.787s
data/downloaded/72/FrontRedDressLabel.jpg: took time: 0.821s
data/downloaded/72/BackRedDressLabel.jpg: took time: 0.74

data/downloaded/129/Omero_front.jpg: took time: 0.741s
data/downloaded/130/RiccioBrand.jpg: took time: 0.998s
data/downloaded/130/RiccioBack.jpg: took time: 6.003s
data/downloaded/130/RiccioStrip.jpg: took time: 0.848s
data/downloaded/131/Chateau_Mangot_05fl_14.jpg: took time: 6.079s
data/downloaded/131/Chateau_Mangot_bl.JPG: took time: 0.744s
data/downloaded/132/sized_PL_Chardonnay_back_USA.jpg: took time: 5.915s
data/downloaded/132/sized_PL_Chardonnay_09_front_USA.jpg: took time: 5.810s
data/downloaded/133/Viticoltori_Raetia_Lagrein_Rose_Front_09.jpg: took time: 1.057s
data/downloaded/133/Viticoltori_Raetia_Lagrein_Rose_09.jpg: took time: 1.060s
data/downloaded/134/Viu_Manent_EC_Malbec_09_br3.jpg: took time: 0.976s
data/downloaded/134/Viu_Manenet_EC_Malbec_09_bk.jpg: took time: 5.801s
data/downloaded/135/coroa_d_ouro_wht_back.jpg: took time: 5.873s
data/downloaded/135/coroa_d_ouro_wht_front.jpg: took time: 5.912s
data/downloaded/135/douro_seal.jpg: took time: 5.894s
data/downloaded/1

data/downloaded/192/MAKREDFRONT.JPG: took time: 0.922s
data/downloaded/192/MAKREDBK.JPG: took time: 6.007s
data/downloaded/193/DHEHRE08_Brand_v1.jpg: took time: 6.081s
data/downloaded/193/DHEHRE08_Back_v1.jpg: took time: 1.107s
data/downloaded/194/NAIA_CONTRA_USA_NEW.tif: took time: 1.111s
data/downloaded/194/NAIA_08_BACK_USA150.tif: took time: 6.822s
data/downloaded/195/AgrunellesCarmin2.jpg: took time: 1.330s
data/downloaded/195/AgrunellesCarmin1.jpg: took time: 5.919s
data/downloaded/196/PL_Back_label_ca_mr.JPG: took time: 5.906s
data/downloaded/197/WVV_2008_Riesling_Label_-_3_75x3_75.jpg: took time: 6.192s
data/downloaded/197/PVT_LABEL_Brandon__Amber_-_4x3.jpg: took time: 0.962s
data/downloaded/198/Isolabella_36_Brachetto_2009_front_TTB_PBI.jpg: took time: 6.114s
data/downloaded/198/Isolabella_36_Brachetto_2009_back_TTB.jpg: took time: 5.908s
data/downloaded/199/cs-front-2.jpg: took time: 5.905s
data/downloaded/199/2008-01_Rear_Label.jpg: took time: 5.900s
data/downloaded/200/Salom

data/downloaded/251/Block559_09_PN_F.jpg: took time: 0.801s
data/downloaded/251/Block559_09_PN_B.jpg: took time: 0.964s
data/downloaded/252/Malbec_071310-07.jpg: took time: 5.901s
data/downloaded/253/Amauve_Seguret_Laurances_07_BL_TTB.jpg: took time: 0.858s
data/downloaded/253/Amauve_Seguret_Laurances_07_FL_TTB.jpg: took time: 1.377s
data/downloaded/254/Prosecco__16_Aug_10.jpg: took time: 8.269s
data/downloaded/254/revised_front_Prosecco_Coll.jpg: took time: 1.221s
data/downloaded/255/Cht_Le_Prieur_cuvee_passion_brand.jpg: took time: 5.637s
data/downloaded/255/Ch_Le_Prieur_2.jpg: took time: 1.574s
data/downloaded/256/07WhiteHawk-F-cola.jpg: took time: 6.021s
data/downloaded/256/07WhiteHawk-B-cola.jpg: took time: 5.638s
data/downloaded/257/pinot_grigio_IGT_front.jpg: took time: 0.804s
data/downloaded/257/Rubini_Pinot_Grigio_IGP_Back_4-13_x_4-6_.JPG: took time: 0.891s
data/downloaded/258/barbaresco_brand.jpg: took time: 0.574s
data/downloaded/258/bsco07paolin.jpg: took time: 0.731s
data/

...

data/downloaded/1455/COLA_130237_Front.jpg: took time: 6.053s
data/downloaded/1455/COLA_130237_Back.jpg: took time: 5.864s
data/downloaded/1456/12_Ran_Pet.jpg: took time: 5.859s
data/downloaded/1457/Inca_Chardonnay_2013_Front_Label.jpg: took time: 0.784s
data/downloaded/1457/Inca_Chardonnay_2013_Back_Label.jpg: took time: 0.970s
data/downloaded/1458/Sequin_CA_Moscato_2012_FL.jpg: took time: 5.744s
data/downloaded/1458/Sequin_CA_Moscato_2012_BL.jpg: took time: 5.801s
data/downloaded/1459/57-Dominio-DE-Requena-0919.jpg: took time: 0.812s
data/downloaded/1459/57_DOMINIO_DE_REQUENA_BRUT.jpg: took time: 5.864s
data/downloaded/1460/VardaMerlotBrand4.jpg: took time: 0.816s
data/downloaded/1460/VardaMerlotFront2.jpg: took time: 5.983s
data/downloaded/1461/OLIVI_I_PURI_FRONT.jpg: took time: 15.042s
data/downloaded/1461/OLIVI_I_PURI_BACK.jpg: took time: 5.899s
data/downloaded/1462/07_NV_Oak_Cab_fr_750ml_Jackie.jpg: took time: 6.062s
data/downloaded/1462/07_NV_Oak_Cab_bk_750ml_Jackie_Healdsburg.j

data/downloaded/1517/tirillaCyL.jpg: took time: 1.780s
data/downloaded/1517/front_CaminoSanPablo_RED_98_mm_x_115_mm.jpg: took time: 6.067s
data/downloaded/1518/CabSauv-Black.jpg: took time: 0.795s
data/downloaded/1518/750BackRes.jpg: took time: 5.956s
data/downloaded/1519/ReduxZinBack.jpg: took time: 6.043s
data/downloaded/1519/ReduxZin25x6.jpg: took time: 1.020s
data/downloaded/1520/BARTHOD-CHAM-MUSIGNY-FR.jpg: took time: 6.213s
data/downloaded/1520/rd-burgundy-comb-new-format.jpg: took time: 6.053s
data/downloaded/1520/BARTHOD-CHAM-MUSIGNY-NECK.jpg: took time: 5.732s
data/downloaded/1521/Pedro_Ximenez_Anada_2011_-_front.jpg: took time: 5.966s
data/downloaded/1521/Pedro_Ximenez_Anada_2011_-_back.jpg: took time: 0.872s
data/downloaded/1521/Alvear_Generic_GW-ImpStrip.jpg: took time: 6.112s
data/downloaded/1522/Petit_ChABLIS.JPG: took time: 0.791s
data/downloaded/1523/RaineyFamilyReserve_cabernet_COLA.jpg: took time: 0.860s
data/downloaded/1524/Burgaud_Cote_du_Py_Javernieres_B.jpg: took 

data/downloaded/1578/gws_import_strip.JPG: took time: 0.782s
data/downloaded/1579/2012_SGP_back.jpg: took time: 0.975s
data/downloaded/1579/2012_SGP_front.jpg: took time: 6.034s
data/downloaded/1580/BennettValleyCellars_2012Pinot_front.jpg: took time: 6.098s
data/downloaded/1580/BennettValleyCellars_2012Pinot_back.jpg: took time: 6.616s
data/downloaded/1581/TapDance_Chard.jpg: took time: 6.046s
data/downloaded/1582/Cristina_Front_2012.jpg: took time: 6.002s
data/downloaded/1582/Cristina_Back_TTB.jpg: took time: 5.911s
data/downloaded/1583/Andrea_BL_CE.jpg: took time: 0.955s
data/downloaded/1583/Andrea_Blanc.jpg: took time: 5.924s
data/downloaded/1584/LN_Chardonnay_2012_front.jpg: took time: 0.803s
data/downloaded/1584/TAMALPAIS_Wine_72x23.jpg: took time: 0.757s
data/downloaded/1584/LN_Chardonnay_2012_back.jpg: took time: 0.958s
data/downloaded/1585/Nibias_FL.jpg: took time: 5.864s
data/downloaded/1585/Nibias_Blanco.jpg: took time: 5.945s
data/downloaded/1586/Baret_rge_09_brdT.jpg: took

data/downloaded/1635/Joaquin_taurasi_back.jpg: took time: 6.010s
data/downloaded/1635/Joaquin_taurasi_front.jpg: took time: 6.034s
data/downloaded/1636/97075_front.jpg: took time: 6.023s
data/downloaded/1636/97075_back.jpg: took time: 5.853s
data/downloaded/1637/Mourvedre_2012_Front-V2_3x4.jpg: took time: 0.886s
data/downloaded/1637/Mourvedre_2012_Back-V2_3x4.jpg: took time: 5.978s
data/downloaded/1638/BT_BIO_DOM_Butler09_750F_NP_060914TTB.jpg: took time: 6.244s
data/downloaded/1638/BT_BIO_DOM_Butler09_750B_NP_041014.jpg: took time: 2.551s
data/downloaded/1639/Clarendelle_Amberwine_brand.jpg: took time: 5.962s
data/downloaded/1639/Clarendelle_Amberwine_back.JPG: took time: 5.986s
data/downloaded/1640/Lex_Front.jpg: took time: 0.722s
data/downloaded/1640/Lex_back.jpg: took time: 5.926s
data/downloaded/1641/Guarini_Negroamaro_Burdi_BL_170dpi.jpg: took time: 6.006s
data/downloaded/1641/Guarini_Negroamaro_Burdi_170dpi.jpg: took time: 5.848s
data/downloaded/1642/aulus_retro.jpg: took time: 

data/downloaded/1695/CHARDONNAY_edit.jpg: took time: 0.917s
data/downloaded/1696/12AR_LG_Format_Mag_COLA.jpg: took time: 0.626s
data/downloaded/1697/G_Lignier_Clos_de_la_Roche.jpg: took time: 5.945s
data/downloaded/1697/Back_Label_2_and_a_half_by_5_inches_-_2011.jpg: took time: 5.847s
data/downloaded/1697/G_Lignier_Neck_Label_2011.jpg: took time: 5.909s
data/downloaded/1698/POENTO--BRANCO.jpg: took time: 6.094s
data/downloaded/1699/FP_2013PN_BRAND.jpg: took time: 0.915s
data/downloaded/1699/FP_2013PN_BACK2.jpg: took time: 6.006s
data/downloaded/1700/Contre_Montlandrie_2009.jpg: took time: 5.696s
data/downloaded/1700/Montlandrie_09.jpg: took time: 0.523s
data/downloaded/1701/Tololo_Sauvignon_Blanc_brand.jpg: took time: 5.809s
data/downloaded/1701/Tololo_Sauvignon_Blanc_back.jpg: took time: 0.893s
data/downloaded/1702/Enoch_RED_Generic_Label.jpg: took time: 1.098s
data/downloaded/1703/CR02_Front_Chablis.jpg: took time: 6.258s
data/downloaded/1703/CR02_back_labels__Chablis.jpg: took time:

In [31]:
# dest= 'data/downloaded/0'
# url1= 'https://img.mediavitis.com/cola_images/Chardonnay_VSR2008.jpg'
# url12 = 'https://img.mediavitis.com/cola_images/backlabel.jpg'

# head_tail = os.path.split(url12)

# head_tail[1]
# frame_name = os.path.join(dest, head_tail[1])
# print(frame_name)

data/downloaded/0/backlabel.jpg


In [32]:
# import os

# import requests

# from time import time

# from multiprocessing.pool import ThreadPool
# from clint.textui import progress

In [41]:
# # r = requests.get(url1, stream = True)

# url = url1
# start = time()
# try:
#     os.makedirs(dest, exist_ok=True)
# #     print("Directory doesn't exist. \nCreating Directory..." )
# except OSError as error:
#     print(error)

# r = requests.get(url, stream = True)
# with open(frame_name, "wb") as f:
#     total_length = int(r.headers.get('content-length'))
#     for chunk in progress.bar(r.iter_content(chunk_size=2391975), expected_size=(total_length/1024) + 1):
#         if chunk:
#             f.write(chunk)
# print(f"{frame_name}: took time: {time() - start :.3f}")
# # with open(path, 'wb') as f:

# #     for ch in r:

# #         f.write(ch)

data/downloaded/0/backlabel.jpg: took time: 0.279
